In [1]:
import pandas as pd
import numpy as np
from typing import Dict, List, Any
import ast
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False
import os
# 获取当前工作目录（通常是 notebooks/）
CURRENT_DIR = os.getcwd()
print("当前工作目录:", CURRENT_DIR)

# 推断项目根目录：假设 notebooks 是子目录
PROJECT_ROOT = os.path.abspath(os.path.join(CURRENT_DIR, '..'))

print("项目根目录:", PROJECT_ROOT)

# 定义输出目录
OUTPUT_DIR = os.path.join(PROJECT_ROOT, 'outputs', 'disrupt')
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 定义数据路径
DATA_DIR = os.path.join(PROJECT_ROOT, 'data', 'raw')
df = pd.read_csv(os.path.join(DATA_DIR, 'top10_journals_data.csv'))

# 定义关键词频率分析器
class KeywordFrequencyAnalyzer:
    """关键词频率分析器 - 提取每个期刊的Top5关键词"""
    
    def __init__(self, df: pd.DataFrame):
        self.df = df
        self.results = {}
    
    def analyze(self, min_papers: int = 5, **kwargs) -> Dict[str, List[str]]:
        """
        分析每个期刊的Top5关键词
        """
        print(" 分析期刊关键词频率...")
        
        # 提取和清洗所有关键词
        paper_keywords = self._extract_all_keywords()
        
        # 按期刊统计词频
        journal_keyword_freq = self._calculate_journal_keyword_freq(paper_keywords, min_papers)
        
        # 保存词频数据供后续使用
        self._journal_keyword_freq = journal_keyword_freq
        
        # 提取每个期刊的Top5关键词
        journal_top_keywords = self._get_top_keywords(journal_keyword_freq)
        
        self.results = journal_top_keywords
        return journal_top_keywords
    
    def _extract_all_keywords(self) -> Dict[str, List[str]]:
        """提取所有论文的关键词"""
        paper_keywords = {}
        
        for idx, row in self.df.iterrows():
            # 使用索引作为paper_id，避免doi空值问题
            paper_id = f"paper_{idx}"
            keywords = []
            
            if pd.notna(row.get('Keywords')):
                try:
                    keywords = self._parse_keyword_field(row['Keywords'])
                except (ValueError, SyntaxError):
                    keywords = []
            
            paper_keywords[paper_id] = keywords
        
        total_keywords = sum(len(keywords) for keywords in paper_keywords.values())
        papers_with_keywords = sum(1 for keywords in paper_keywords.values() if keywords)
        
        print(f"总论文数: {len(paper_keywords)}")
        print(f"有关键词的论文: {papers_with_keywords}")
        print(f"总关键词数: {total_keywords}")
        
        return paper_keywords
    
    def _parse_keyword_field(self, word_field) -> List[str]:
        """解析word字段"""
        if pd.isna(word_field):
            return []
        
        if isinstance(word_field, str):
            if word_field.startswith('['):
                try:
                    keywords = ast.literal_eval(word_field)
                except:
                    keywords = [word_field]
            else:
                if ';' in word_field:
                    keywords = [kw.strip() for kw in word_field.split(';') if kw.strip()]
                else:
                    keywords = [kw.strip() for kw in word_field.split(',') if kw.strip()]
        elif isinstance(word_field, list):
            keywords = word_field
        else:
            keywords = [str(word_field)]
        
        cleaned_keywords = [str(kw).lower().strip() for kw in keywords if str(kw).strip()]
        return cleaned_keywords
    
    def _calculate_journal_keyword_freq(self, paper_keywords: Dict[str, List[str]], min_papers: int) -> Dict[str, Counter]:
        """按期刊统计关键词频率"""
        journal_keywords = {}
        
        for paper_id, keywords in paper_keywords.items():
            # 直接从索引获取期刊信息
            idx = int(paper_id.split('_')[1])
            journal = self.df.iloc[idx]['Source Title']
            
            if pd.isna(journal):
                continue
            
            if journal not in journal_keywords:
                journal_keywords[journal] = []
            
            journal_keywords[journal].extend(keywords)
        
        # 过滤小期刊
        journal_keyword_freq = {}
        for journal, keywords in journal_keywords.items():
            journal_papers = self.df[self.df['Source Title'] == journal]
            if len(journal_papers) >= min_papers and keywords:
                journal_keyword_freq[journal] = Counter(keywords)
        
        print(f"分析的期刊数: {len(journal_keyword_freq)}")
        return journal_keyword_freq
    
    def _get_top_keywords(self, journal_keyword_freq: Dict[str, Counter]) -> Dict[str, List[str]]:
        """提取每个期刊的Top5关键词"""
        journal_top_keywords = {}
        for journal, keyword_counter in journal_keyword_freq.items():
            top_keywords = keyword_counter.most_common(5)
            journal_top_keywords[journal] = [keyword for keyword, count in top_keywords]
        
        return journal_top_keywords

# 执行分析
print("开始关键词频率分析...")
analyzer = KeywordFrequencyAnalyzer(df)
journal_top_keywords = analyzer.analyze(min_papers=5)

# 显示结果
print(f"\n 期刊主题分析完成: {len(journal_top_keywords)} 种期刊")
# 显示所有期刊的Top5关键词
print("\n 期刊主题分布 (所有期刊):")
print("=" * 80)

for i, (journal, keywords) in enumerate(sorted(journal_top_keywords.items(), key=lambda x: len(x[1]), reverse=True), 1):
    print(f"{i:2d}. {journal}:")
    for j, keyword in enumerate(keywords, 1):
        print(f"    {j}. {keyword}")
    print()  # 空行分隔

print(f"总共分析了 {len(journal_top_keywords)} 种期刊")

当前工作目录: c:\Users\28623\OneDrive\Desktop\BankJournalAnalysis\notebooks
项目根目录: c:\Users\28623\OneDrive\Desktop\BankJournalAnalysis
开始关键词频率分析...
 分析期刊关键词频率...
总论文数: 9747
有关键词的论文: 7293
总关键词数: 42534
分析的期刊数: 10

 期刊主题分析完成: 10 种期刊

 期刊主题分布 (所有期刊):
 1. BIOMEDICAL ENGINEERING ONLINE:
    1. model
    2. system
    3. classification
    4. diagnosis
    5. segmentation

 2. IEEE PULSE:
    1. pressure regulatory system
    2. control parameters
    3. cells
    4. hepatocellular-carcinoma
    5. humans

 3. IEEE TRANSACTIONS ON BIOMEDICAL ENGINEERING:
    1. model
    2. system
    3. classification
    4. design
    5. algorithm

 4. NATURE BIOMEDICAL ENGINEERING:
    1. expression
    2. cancer
    3. in-vivo
    4. activation
    5. delivery

 5. MEDICAL ENGINEERING & PHYSICS:
    1. model
    2. system
    3. mechanical-properties
    4. pressure
    5. validation

 6. BIOMEDICAL SIGNAL PROCESSING AND CONTROL:
    1. classification
    2. diagnosis
    3. algorithm
    4. model
    5. system

In [ ]:
import json
import requests

# 请替换为你的 API Key
api_key = "Bearer cyjdtVYXSGWgwiUdnLMs:DvKIMQbkHgKlYljNcbhN"  # 你的实际API Key
url = "https://spark-api-open.xf-yun.com/v2/chat/completions"

def analyze_journal_topics(journal_top_keywords):
    """分析期刊主题"""
    print("开始分析期刊主题...")
    
    results = {}
    
    for journal, keywords in journal_top_keywords.items():
        print(f"\n分析期刊: {journal}")
        
        # 构建提示词
        prompt = f"""请根据这个学术期刊的名称和其高频关键词，简要分析该期刊的主要研究方向和主题侧重点。

期刊名称：{journal}
高频关键词：{', '.join(keywords)}

请用100字左右描述该期刊的研究主题特点、技术方法和应用领域。直接输出分析结果，不要有任何思维过程。"""
        
        # 构建消息格式
        message = [
            {
                "role": "user", 
                "content": prompt
            }
        ]
        
        try:
            analysis = get_answer(message)
            results[journal] = analysis
            print(f"✅ 分析完成")
        except Exception as e:
            print(f"❌ 分析失败: {e}")
            results[journal] = "分析失败"
    
    return results

def get_answer(message):
    """调用星火API获取回答"""
    headers = {
        'Authorization': api_key,
        'content-type': "application/json"
    }
    body = {
        "model": "x1",
        "user": "journal_analyzer",
        "messages": message,
        "stream": True,
        "max_tokens": 1024,
        "temperature": 0.7,
        "reasoning": False  # 关键：完全禁用思维链
    }
    
    full_response = ""

    response = requests.post(url=url, json=body, headers=headers, stream=True)
    
    for chunks in response.iter_lines():
        if chunks and b'[DONE]' not in chunks:
            try:
                # 处理数据格式
                if chunks.startswith(b'data: '):
                    data_org = chunks[6:]
                else:
                    data_org = chunks
                
                chunk = json.loads(data_org)
                text = chunk['choices'][0]['delta']
                
                # 只收集最终回复
                if 'content' in text and text['content']:
                    content = text["content"]
                    print(content, end="")
                    full_response += content
                    
            except Exception as e:
                continue
    
    return full_response

# 直接使用你的journal_top_keywords数据
if __name__ == '__main__':
    # 分析期刊主题
    results = analyze_journal_topics(journal_top_keywords)

开始分析期刊主题...

分析期刊: BIOMEDICAL ENGINEERING ONLINE
该刊聚焦生物医学工程领域的数字化解决方案，以**计算建模**为核心，运用**机器学习分类算法**与**医学影像分割技术**，致力于构建智能化**诊断系统**。研究覆盖从器官级仿真到临床决策支持的多层次体系，侧重将工程技术转化为可落地的健康评估工具，应用于肿瘤检测、病理分级及个性化治疗规划等场景。✅ 分析完成

分析期刊: IEEE PULSE


KeyboardInterrupt: 

In [ ]:
import json

# 定义输出目录：项目根目录下的 outputs/theme
OUTPUT_THEME_DIR = os.path.join(PROJECT_ROOT, 'outputs', 'theme')
os.makedirs(OUTPUT_THEME_DIR, exist_ok=True)

# 输出文件路径
keywords_output_path = os.path.join(OUTPUT_THEME_DIR, 'journal_keywords.json')
analysis_output_path = os.path.join(OUTPUT_THEME_DIR, 'journal_analysis.json')

# 准备元数据
metadata = {
    "source_file": "top10_journals_data.csv",
    "total_journals": len(journal_top_keywords)
}

# === 输出 1: Top5 关键词表 ===
keywords_data = {
    "metadata": metadata,
    "data": journal_top_keywords  # {期刊: [Top5关键词]}
}

with open(keywords_output_path, 'w', encoding='utf-8') as f:
    json.dump(keywords_data, f, ensure_ascii=False, indent=2)

print(f"✅ 期刊关键词表已保存至:\n   {keywords_output_path}")

# === 输出 2: AI 主题分析结果 ===
analysis_data = {
    "metadata": metadata,
    "data": results  # {期刊: "AI分析文本"}
}

with open(analysis_output_path, 'w', encoding='utf-8') as f:
    json.dump(analysis_data, f, ensure_ascii=False, indent=2)

print(f"✅ 期刊主题分析结果已保存至:\n   {analysis_output_path}")


开始分析期刊主题...

分析期刊: BIOMEDICAL ENGINEERING ONLINE
该期刊聚焦生物医学工程领域的计算建模与智能系统开发，以机器学习驱动的疾病诊断为核心，重点研究医学影像自动分割、生物信号分类算法及临床决策支持系统。技术方法涵盖深度学习模型构建、多模态数据融合与模式识别，应用于肿瘤检测、神经疾病分析和手术导航等精准医疗场景，强调算法创新与临床转化的结合。✅ 分析完成

分析期刊: IEEE PULSE
IEEE PULSE聚焦生物医学工程与临床医学交叉领域，主要研究压力调控系统在生理功能优化及疾病治疗中的应用。其核心方向包括闭环控制系统设计、生物参数动态监测技术，尤其关注肝细胞癌微环境压力调控机制及抗癌疗法开发。技术手段涵盖生物信号传感、非线性控制算法建模及细胞级病理模型构建，应用场景集中于智能医疗设备研发、精准肿瘤治疗及人体生理机能干预，体现从分子机制探索到临床转化的创新路径。✅ 分析完成

分析期刊: IEEE TRANSACTIONS ON BIOMEDICAL ENGINEERING
该期刊聚焦生物医学工程领域的技术创新，以建模、系统设计、分类方法和算法开发为核心，研究智能医疗设备、生理信号处理及健康监测系统。侧重运用机器学习、数据分析和工程优化手段解决临床医学、康复工程及生物信息学中的技术难题，推动精准诊疗与个性化医疗发展。✅ 分析完成

分析期刊: NATURE BIOMEDICAL ENGINEERING
该期刊聚焦于生物医学工程在癌症研究中的创新应用，以基因/蛋白**表达**调控为核心，结合**体内**（in-vivo）实验验证，探索靶向**癌症**治疗的**递送**系统设计与**激活**机制。主要采用纳米技术、基因编辑及智能材料等手段，开发精准诊疗技术，侧重从分子层面实现药物可控释放与肿瘤微环境响应，推动个性化抗癌疗法转化。✅ 分析完成

分析期刊: MEDICAL ENGINEERING & PHYSICS
该期刊聚焦医学工程与物理原理的交叉应用，以建模（model）、系统（system）为核心方法，侧重生物材料机械性能（mechanical-properties）、压力（pressure）参数分析及技术验证（validation）。主要研究涵盖医疗设备创新设计、生物力学仿真、诊疗系统开发，涉及流体动力学、组织力学等领